In [4]:
# Download and install Miniconda
!wget -q https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
!chmod +x Miniconda3-latest-Linux-x86_64.sh
!bash Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local

# Update conda
!eval "$(/usr/local/bin/conda shell.bash hook)" && conda update -y -n base -c defaults conda

# (Optional) Append site-packages for later Python access (matches your logic)
import sys
sys.path.append('/usr/local/lib/python3.10/site-packages')

# Create the Python 3.10 conda environment
!eval "$(/usr/local/bin/conda shell.bash hook)" && conda create -y -n myenv python=3.10


PREFIX=/usr/local
Unpacking payload ...
entry_point.py:256: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
entry_point.py:256: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.

Installing base environment...

Preparing transaction: ...working... done
Executing transaction: ...working... done
entry_point.py:256: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
installation finished.
    You currently have a PYTHONPATH environment variable set. This may cause
    unexpected behavior when running the Python interpreter in Miniconda3.
    For best results, please verify that your PYTHONPATH only points to
    directories o

In [5]:
%%bash
set -e

# Load conda into shell
source /usr/local/etc/profile.d/conda.sh
conda activate myenv

# Install cudatoolkit and pip packages (exact versions)
conda install -y cudatoolkit=11.3
pip install -q opennyai "numpy<2" cupy-cuda113

# Run GPU test
python -c "
import spacy
import cupy
print('spaCy GPU support:', spacy.prefer_gpu())
print('CuPy GPU device:', cupy.cuda.runtime.getDevice())
"


Channels:
 - defaults
Platform: linux-64
Solving environment: done

## Package Plan ##

  environment location: /usr/local/envs/myenv

  added / updated specs:
    - cudatoolkit=11.3


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    cudatoolkit-11.3.1         |       h2bc3f7f_2       549.3 MB
    ------------------------------------------------------------
                                           Total:       549.3 MB

The following NEW packages will be INSTALLED:

  cudatoolkit        pkgs/main/linux-64::cudatoolkit-11.3.1-h2bc3f7f_2 



 done                                                
Preparing transaction: done
Verifying transaction: done
Executing transaction: | By downloading and using the CUDA Toolkit conda packages, you accept the terms and conditions of the CUDA End User License Agreement (EULA): https://docs.nvidia.com/cuda/eula/index.html

done
   ━━━━━━━━━━━━━━━━━━━━━━━

/usr/local/envs/myenv/lib/python3.10/site-packages/torch/__init__.py:614: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at ../torch/csrc/tensor/python_tensor.cpp:451.)
  _C._set_default_tensor_type(t)


In [19]:
# with open("run_inference.py", "w") as f:
#     f.write('''\
# import os
# import json
# from glob import glob
# from opennyai import Pipeline
# from opennyai.utils import Data
# import spacy

# spacy.require_gpu()

# input_dir = "/kaggle/working/DHARA/cases"
# output_dir = "rrl"
# os.makedirs(output_dir, exist_ok=True)

# file_paths = sorted(glob(os.path.join(input_dir, "*.txt")))
# texts = []
# doc_ids = []

# for path in file_paths:
#     with open(path, 'r', encoding='utf-8') as f:
#         texts.append(f.read())
#         doc_ids.append(os.path.splitext(os.path.basename(path))[0])

# print(f"📄 Loaded {len(texts)} documents...", flush=True)

# data = Data(texts, use_gpu=True)
# pipe = Pipeline(components=["Rhetorical_Role"], use_gpu=True)
# results = pipe(data)

# for doc_id, result in zip(doc_ids, results):
#     output_path = os.path.join(output_dir, f"{doc_id}.json")
#     with open(output_path, "w", encoding="utf-8") as f:
#         json.dump(result, f, ensure_ascii=False, indent=2)
#     print(f"✅ Saved: {output_path}", flush=True)

# print(f"🏁 Inference complete. {len(results)} files saved to {output_dir}", flush=True)
# ''')

with open("run_inference.py", "w") as f:
    f.write('''\
import os
import json
from glob import glob
from opennyai import Pipeline
from opennyai.utils import Data
import spacy

spacy.require_gpu()

input_dir = "/kaggle/working/DHARA/cases"
output_dir = "rrl"
os.makedirs(output_dir, exist_ok=True)

# Load only case90 to case99
file_paths = [
    os.path.join(input_dir, f"case{i}.txt") for i in range(90, 100)
    if os.path.exists(os.path.join(input_dir, f"case{i}.txt"))
]
print(file_paths)

texts = []
doc_ids = []

for path in file_paths:
    with open(path, 'r', encoding='utf-8') as f:
        texts.append(f.read())
        doc_ids.append(os.path.splitext(os.path.basename(path))[0])

print(f"📄 Loaded {len(texts)} documents (case90–case99)...", flush=True)

data = Data(texts, use_gpu=True)
pipe = Pipeline(components=["Rhetorical_Role"], use_gpu=True)
results = pipe(data)

for doc_id, result in zip(doc_ids, results):
    output_path = os.path.join(output_dir, f"{doc_id}.json")
    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(result, f, ensure_ascii=False, indent=2)
    print(f"✅ Saved: {output_path}", flush=True)

print(f"🏁 Inference complete. {len(results)} files saved to {output_dir}", flush=True)
''')


In [20]:
import subprocess

# Path to the Conda env’s Python
python_path = "/usr/local/envs/myenv/bin/python"

# Run script with live logs
process = subprocess.Popen(
    [python_path, "run_inference.py"],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    universal_newlines=True,
    bufsize=1,
)

for line in process.stdout:
    print(line, end="")  # Prints each line live

process.wait()


/usr/local/envs/myenv/lib/python3.10/site-packages/torch/__init__.py:614: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at ../torch/csrc/tensor/python_tensor.cpp:451.)
  _C._set_default_tensor_type(t)
['/kaggle/working/DHARA/cases/case90.txt', '/kaggle/working/DHARA/cases/case91.txt', '/kaggle/working/DHARA/cases/case92.txt', '/kaggle/working/DHARA/cases/case93.txt', '/kaggle/working/DHARA/cases/case94.txt', '/kaggle/working/DHARA/cases/case95.txt', '/kaggle/working/DHARA/cases/case96.txt', '/kaggle/working/DHARA/cases/case97.txt', '/kaggle/working/DHARA/cases/case98.txt', '/kaggle/working/DHARA/cases/case99.txt']
📄 Loaded 10 documents (case90–case99)...
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predi

0

In [22]:
!zip -r /kaggle/working/rrl_final.zip /kaggle/working/rrl


  adding: kaggle/working/rrl/ (stored 0%)
  adding: kaggle/working/rrl/case139.json (deflated 74%)
  adding: kaggle/working/rrl/case2.json (deflated 75%)
  adding: kaggle/working/rrl/case25.json (deflated 72%)
  adding: kaggle/working/rrl/case93.json (deflated 72%)
  adding: kaggle/working/rrl/case361.json (deflated 80%)
  adding: kaggle/working/rrl/case55.json (deflated 73%)
  adding: kaggle/working/rrl/case245.json (deflated 74%)
  adding: kaggle/working/rrl/case215.json (deflated 74%)
  adding: kaggle/working/rrl/case41.json (deflated 75%)
  adding: kaggle/working/rrl/case336.json (deflated 77%)
  adding: kaggle/working/rrl/case135.json (deflated 84%)
  adding: kaggle/working/rrl/case376.json (deflated 73%)
  adding: kaggle/working/rrl/case311.json (deflated 78%)
  adding: kaggle/working/rrl/case297.json (deflated 75%)
  adding: kaggle/working/rrl/case141.json (deflated 74%)
  adding: kaggle/working/rrl/case199.json (deflated 72%)
  adding: kaggle/working/rrl/case356.json (deflated 

In [16]:
%%bash
set -e
eval "$(conda shell.bash hook)"
conda activate myenv

# Run Python file using conda env interpreter
python run_inference.py

Process is terminated.


In [14]:
%%bash
set -e

# Activate conda env
source /usr/local/etc/profile.d/conda.sh
conda activate myenv

# Install necessary libraries (exact versions)
conda install -y cudatoolkit=11.3
pip install -q opennyai "numpy<2" cupy-cuda113 "spacy[cuda113]"

# Run your logic
PYTHONUNBUFFERED=1 python - <<EOF
import os
import json
from glob import glob
from opennyai import Pipeline
from opennyai.utils import Data
import sys

import spacy
spacy.require_gpu()

sys.stdout.reconfigure(line_buffering=True)


# Paths
input_dir = "/kaggle/working/DHARA/cases"
output_dir = "/kaggle/working/DHARA/rrl"
os.makedirs(output_dir, exist_ok=True)

# 1. Collect and read all .txt case files
file_paths = sorted(glob(os.path.join(input_dir, "*.txt")))
texts = []
doc_ids = []

for path in file_paths:
    with open(path, 'r', encoding='utf-8') as f:
        texts.append(f.read())
        doc_ids.append(os.path.splitext(os.path.basename(path))[0])

# 2. Wrap with OpenNyAI-compatible Data object
data = Data(texts, use_gpu=True)

# 3. Init pipeline (use GPU if available)
pipe = Pipeline(components=["Rhetorical_Role"], use_gpu=True)

# 4. Run RRL
results = pipe(data)

# 5. Save each result as individual JSON file
for doc_id, result in zip(doc_ids, results):
    output_path = os.path.join(output_dir, f"{doc_id}.json")
    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(result, f, ensure_ascii=False, indent=2)

print(f"✅ Inference complete. {len(results)} files saved to {output_dir}")
EOF


Process is terminated.


In [8]:
!git clone https://github.com/AnshSinghal/DHARA.git


Cloning into 'DHARA'...
remote: Enumerating objects: 810, done.
remote: Counting objects: 100% (810/810), done.
remote: Compressing objects: 100% (800/800), done.
remote: Total 810 (delta 9), reused 809 (delta 8), pack-reused 0 (from 0)
Receiving objects: 100% (810/810), 11.82 MiB | 13.34 MiB/s, done.
Resolving deltas: 100% (9/9), done.


In [1]:
# Solution 1: Force install cupy-cuda12x with specific CUDA path
!curl --proto '=https' --tlsv1.2 -sSf https://sh.rustup.rs | sh -s -- -y
import os
os.environ['PATH'] += ":/root/.cargo/bin"

# Set CUDA environment variables before installing packages
os.environ['CUDA_HOME'] = '/usr/local/cuda'
os.environ['CUDA_ROOT'] = '/usr/local/cuda'
os.environ['LD_LIBRARY_PATH'] = '/usr/local/cuda/lib64:/usr/local/cuda/lib'
os.environ['PATH'] += ':/usr/local/cuda/bin'

# Configure ldconfig
!echo "/usr/local/cuda/lib64" >> /etc/ld.so.conf.d/cuda.conf
!ldconfig

# Set up uv with explicit CUDA environment
!uv python install 3.10
!uv python pin 3.10
!uv init --python 3.10

# Install packages with CUDA environment variables
!CUDA_HOME=/usr/local/cuda LD_LIBRARY_PATH=/usr/local/cuda/lib64 uv add -q opennyai "numpy<2"

# Install cupy first with explicit CUDA version
!CUDA_HOME=/usr/local/cuda LD_LIBRARY_PATH=/usr/local/cuda/lib64 uv add -q "cupy-cuda12x"

# Then install spacy with CUDA support
!CUDA_HOME=/usr/local/cuda LD_LIBRARY_PATH=/usr/local/cuda/lib64 uv add -q "spacy[cuda12x]"

import subprocess

# Test script with environment variables
python_code = """
import os
import sys

# Set CUDA environment variables in the Python process
os.environ['CUDA_HOME'] = '/usr/local/cuda'
os.environ['LD_LIBRARY_PATH'] = '/usr/local/cuda/lib64:/usr/local/cuda/lib'
os.environ['PATH'] += ':/usr/local/cuda/bin'

# Print environment info
print("Python executable:", sys.executable)
print("CUDA_HOME:", os.environ.get('CUDA_HOME', 'Not set'))
print("LD_LIBRARY_PATH:", os.environ.get('LD_LIBRARY_PATH', 'Not set'))

try:
    import cupy
    print("CuPy version:", cupy.__version__)
    print("CUDA version from CuPy:", cupy.cuda.runtime.runtimeGetVersion())
    print("Available GPUs:", cupy.cuda.runtime.getDeviceCount())
except Exception as e:
    print("CuPy error:", e)

try:
    import spacy
    print("spaCy version:", spacy.__version__)
    
    # Try to enable GPU
    is_gpu_available = spacy.prefer_gpu()
    print(f"spaCy GPU support: {is_gpu_available}")
    
    if is_gpu_available:
        gpu_id = cupy.cuda.runtime.getDevice()
        print(f"✅ spaCy is now using GPU ID: {gpu_id}")
    else:
        print("❌ GPU support for spaCy is still not available.")
        
except Exception as e:
    print("spaCy error:", e)
"""

# Run with explicit environment
env = os.environ.copy()
env.update({
    'CUDA_HOME': '/usr/local/cuda',
    'LD_LIBRARY_PATH': '/usr/local/cuda/lib64:/usr/local/cuda/lib',
    'PATH': env['PATH'] + ':/usr/local/cuda/bin'
})

completed_process = subprocess.run(
    ["uv", "run", "python", "-c", python_code], 
    capture_output=True, 
    text=True, 
    env=env
)

print("--- Script Output ---")
print(completed_process.stdout)
if completed_process.stderr:
    print("--- Errors ---")
    print(completed_process.stderr)


# Alternative Solution 2: If above doesn't work, try installing from source
print("\n--- Alternative Solution 2: Install from source ---")

alternative_code = """
# Alternative approach - install cupy from source with specific CUDA version
!CUDA_HOME=/usr/local/cuda LD_LIBRARY_PATH=/usr/local/cuda/lib64 uv remove cupy-cuda12x
!CUDA_HOME=/usr/local/cuda LD_LIBRARY_PATH=/usr/local/cuda/lib64 uv add -q --no-cache cupy

# Or try with pip inside uv environment
!uv run pip install --no-cache-dir --force-reinstall cupy-cuda12x

# Test again
python_test = '''
import os
os.environ['CUDA_HOME'] = '/usr/local/cuda'
os.environ['LD_LIBRARY_PATH'] = '/usr/local/cuda/lib64'

import spacy
import cupy

is_gpu_available = spacy.prefer_gpu()
print(f"spaCy GPU support: {is_gpu_available}")

if is_gpu_available:
    gpu_id = cupy.cuda.runtime.getDevice()
    print(f"✅ spaCy is now using GPU ID: {gpu_id}")
else:
    print("❌ GPU support for spaCy is still not available.")
'''

!uv run python -c "$python_test"
"""

print("If the first solution doesn't work, try this alternative:")
print(alternative_code)


# Solution 3: Manual cupy installation with specific CUDA version
print("\n--- Solution 3: Manual CUDA environment setup ---")

solution3_code = """
# Remove existing installations and reinstall with proper CUDA setup
!uv remove spacy cupy-cuda12x
!uv sync

# Set up CUDA environment more explicitly
import os
os.environ['CUDA_HOME'] = '/usr/local/cuda'
os.environ['CUDA_ROOT'] = '/usr/local/cuda'
os.environ['CUDA_PATH'] = '/usr/local/cuda'
os.environ['LD_LIBRARY_PATH'] = '/usr/local/cuda/lib64:/usr/local/cuda/lib:/usr/local/cuda/extras/CUPTI/lib64'
os.environ['PATH'] += ':/usr/local/cuda/bin'

# Install with explicit CUDA version
!CUDA_HOME=/usr/local/cuda CUDA_ROOT=/usr/local/cuda LD_LIBRARY_PATH=/usr/local/cuda/lib64 uv add cupy==12.3.0

# Install spacy after cupy is properly installed
!CUDA_HOME=/usr/local/cuda CUDA_ROOT=/usr/local/cuda LD_LIBRARY_PATH=/usr/local/cuda/lib64 uv add "spacy[cuda12x]"

# Test with proper environment
test_script = '''
import os
os.environ['CUDA_HOME'] = '/usr/local/cuda'
os.environ['LD_LIBRARY_PATH'] = '/usr/local/cuda/lib64'

try:
    import cupy as cp
    print("CuPy CUDA version:", cp.cuda.runtime.runtimeGetVersion())
    
    # Test basic GPU operation
    a = cp.array([1, 2, 3, 4, 5])
    print("CuPy GPU test successful:", a.sum())
    
    import spacy
    
    # This should now work
    gpu_available = spacy.prefer_gpu()
    print(f"spaCy GPU available: {gpu_available}")
    
    if gpu_available:
        print("✅ Success! spaCy is using GPU")
    else:
        print("❌ spaCy still not using GPU")
        
except Exception as e:
    print("Error:", e)
    import traceback
    traceback.print_exc()
'''

!uv run python -c "$test_script"
"""

print("Try this more comprehensive solution:")
print(solution3_code)

info: downloading installer
info: profile set to 'default'
info: default host triple is x86_64-unknown-linux-gnu
info: syncing channel updates for 'stable-x86_64-unknown-linux-gnu'
info: latest update on 2025-06-26, rust version 1.88.0 (6b00bc388 2025-06-23)
info: downloading component 'cargo'
info: downloading component 'clippy'
info: downloading component 'rust-docs'
info: downloading component 'rust-std'
info: downloading component 'rustc'
info: downloading component 'rustfmt'
info: installing component 'cargo'
info: installing component 'clippy'
info: installing component 'rust-docs'
 20.1 MiB /  20.1 MiB (100 %)   6.7 MiB/s in  3s         
info: installing component 'rust-std'
 29.5 MiB /  29.5 MiB (100 %)  11.0 MiB/s in  2s         
info: installing component 'rustc'
 76.3 MiB /  76.3 MiB (100 %)  11.8 MiB/s in  6s         
info: installing component 'rustfmt'
info: default toolchain set to 'stable-x86_64-unknown-linux-gnu'

  stable-x86_64-unknown-linux-gnu installed - rustc 1.8

In [3]:
!CUDA_HOME=/usr/local/cuda LD_LIBRARY_PATH=/usr/local/cuda/lib64 uv remove cupy-cuda12x
!CUDA_HOME=/usr/local/cuda LD_LIBRARY_PATH=/usr/local/cuda/lib64 uv add -q --no-cache cupy

# Or try with pip inside uv environment
!uv run pip install --no-cache-dir --force-reinstall cupy-cuda12x

# Test again
python_test = '''
import os
os.environ['CUDA_HOME'] = '/usr/local/cuda'
os.environ['LD_LIBRARY_PATH'] = '/usr/local/cuda/lib64'

import spacy
import cupy

is_gpu_available = spacy.prefer_gpu()
print(f"spaCy GPU support: {is_gpu_available}")

if is_gpu_available:
    gpu_id = cupy.cuda.runtime.getDevice()
    print(f"✅ spaCy is now using GPU ID: {gpu_id}")
else:
    print("❌ GPU support for spaCy is still not available.")
'''

!uv run python -c "$python_test"


Resolved 94 packages in 65ms                                         
Uninstalled 2 packages in 115ms
 - cupy-cuda12x==13.5.1
 - fastrlock==0.8.3
  × Failed to build `cupy==13.5.1`
  ├─▶ The build backend returned an error
  ╰─▶ Call to `setuptools.build_meta.build_wheel` failed (exit status: 1)

      [stdout]
      Clearing directory:
      /tmp/.tmpCqfom2/sdists-v9/pypi/cupy/13.5.1/jFHNDpFg7OJ531j29_2zp/src/cupy/.data
      Generating CUPY_CACHE_KEY from header files...
      CUPY_CACHE_KEY (1729 files matching
      /tmp/.tmpCqfom2/sdists-v9/pypi/cupy/13.5.1/jFHNDpFg7OJ531j29_2zp/src/cupy/_core/include/**):
      689d0ce84be1f21385336456dfa877e7e48b2366

      -------- Configuring Module: cuda --------

      -------- Configuring Module: cusolver --------

      -------- Configuring Module: cudnn --------

      -------- Configuring Module: nccl --------

      -------- Configuring Module: nvtx --------

      -------- Configuring Module: cutensor --------
      command '/usr/bin/c

In [4]:
!uv remove spacy cupy-cuda12x
!uv sync

# Set up CUDA environment more explicitly
import os
os.environ['CUDA_HOME'] = '/usr/local/cuda'
os.environ['CUDA_ROOT'] = '/usr/local/cuda'
os.environ['CUDA_PATH'] = '/usr/local/cuda'
os.environ['LD_LIBRARY_PATH'] = '/usr/local/cuda/lib64:/usr/local/cuda/lib:/usr/local/cuda/extras/CUPTI/lib64'
os.environ['PATH'] += ':/usr/local/cuda/bin'

# Install with explicit CUDA version
!CUDA_HOME=/usr/local/cuda CUDA_ROOT=/usr/local/cuda LD_LIBRARY_PATH=/usr/local/cuda/lib64 uv add cupy==12.3.0

# Install spacy after cupy is properly installed
!CUDA_HOME=/usr/local/cuda CUDA_ROOT=/usr/local/cuda LD_LIBRARY_PATH=/usr/local/cuda/lib64 uv add "spacy[cuda12x]"

# Test with proper environment
test_script = '''
import os
os.environ['CUDA_HOME'] = '/usr/local/cuda'
os.environ['LD_LIBRARY_PATH'] = '/usr/local/cuda/lib64'

try:
    import cupy as cp
    print("CuPy CUDA version:", cp.cuda.runtime.runtimeGetVersion())
    
    # Test basic GPU operation
    a = cp.array([1, 2, 3, 4, 5])
    print("CuPy GPU test successful:", a.sum())
    
    import spacy
    
    # This should now work
    gpu_available = spacy.prefer_gpu()
    print(f"spaCy GPU available: {gpu_available}")
    
    if gpu_available:
        print("✅ Success! spaCy is using GPU")
    else:
        print("❌ spaCy still not using GPU")
        
except Exception as e:
    print("Error:", e)
    import traceback
    traceback.print_exc()
'''

!uv run python -c "$test_script"

error: The dependency `cupy-cuda12x` could not be found in `project.dependencies`
Resolved 94 packages in 0.80ms
Audited 90 packages in 0.03ms
  × Failed to build `cupy==12.3.0`                                                  
  ├─▶ The build backend returned an error
  ╰─▶ Call to
      `setuptools.build_meta:__legacy__.prepare_metadata_for_build_wheel`
      failed (exit status: 1)

      [stdout]
      Clearing directory:
      /root/.cache/uv/sdists-v9/pypi/cupy/12.3.0/GRf_ch1elRNviAZb-btia/src/cupy/.data

      -------- Configuring Module: cuda --------
      Cannot build a stub file.
      Original error: command '/usr/bin/c++' failed with exit code 1

      ************************************************************
      * CuPy Configuration Summary                               *
      ************************************************************

      Build Environment:
        Include directories: ['/usr/local/cuda/include/cub',
      '/root/.cache/uv/sdists-v9/pypi/cupy/1

In [5]:
!strace -e trace=open,openat -o /tmp/uv_spacy_trace.txt uv run python -c "import spacy; spacy.require_gpu()"

/bin/bash: line 1: strace: command not found


In [7]:
import subprocess
import os

# Get the path to nvcc
nvcc_path = subprocess.check_output(['which', 'nvcc']).decode().strip()

# Get CUDA lib path by trimming the path appropriately
cuda_lib_path = os.path.join(os.path.dirname(os.path.dirname(nvcc_path)), 'lib64')

print("CUDA_LIB_PATH:", cuda_lib_path)


CUDA_LIB_PATH: /usr/local/cuda/lib64


In [8]:
# The core of the solution:
# Use the previously discovered CUDA_LIB_PATH to set LD_LIBRARY_PATH
# for the duration of the 'uv run' command.
!LD_LIBRARY_PATH={CUDA_LIB_PATH} uv run python your_script.py

/kaggle/working/.venv/bin/python3: can't open file '/kaggle/working/your_script.py': [Errno 2] No such file or directory


In [ ]:
import os
import subprocess

# --- Step 1: Dynamically locate the CUDA library path ---
# This is more robust than hardcoding the path.
try:
    nvcc_path = subprocess.check_output("which nvcc", shell=True, text=True).strip()
    cuda_root_path = os.path.dirname(os.path.dirname(nvcc_path))
    CUDA_LIB_PATH = os.path.join(cuda_root_path, "lib64")
    print(f"SUCCESS: Discovered CUDA library path: {CUDA_LIB_PATH}")
except (subprocess.CalledProcessError, FileNotFoundError):
    print("ERROR: nvcc not found. Cannot determine CUDA library path.")
    CUDA_LIB_PATH = None

if CUDA_LIB_PATH:
    # --- Step 2: Install dependencies with uv ---
    print("\n--- Installing dependencies with uv for Python 3.10 ---")
    install_command = (
        'uv pip install "spacy[cuda12x]==3.7.4" "cupy-cuda12x==12.3.0" '
        '"torch==2.1.2" "torchaudio==2.1.2" --python 3.10'
    )
    subprocess.run(install_command, shell=True, check=True)
    print("--- Installation complete ---")

    # --- Step 3: Create a verification script ---
    verify_script_content = """
import torch
import spacy
import cupy
import os

print(f"--- Verification inside uv run environment ---")
print(f"LD_LIBRARY_PATH: {os.environ.get('LD_LIBRARY_PATH', 'Not Set')}")
print("-" * 40)

# Verify PyTorch
print(f"PyTorch version: {torch.__version__}")
print(f"Torch CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"Torch device: {torch.cuda.get_device_name(0)}")

print("-" * 40)

# Verify spaCy and CuPy
print(f"spaCy version: {spacy.__version__}")
print(f"CuPy version: {cupy.__version__}")
try:
    # spacy.require_gpu() returns True if a GPU is available, otherwise raises an error.
    spacy.require_gpu(0)
    print("spacy.require_gpu(0) check PASSED.")
    # Further test by moving a small pipeline to the GPU
    nlp = spacy.blank("en")
    nlp.add_pipe("textcat")
    nlp.to_gpu(0)
    print("spaCy model successfully moved to GPU.")
    nlp.to_cpu() # Clean up GPU memory
    print("SUCCESS: spaCy GPU functionality is working correctly.")
except Exception as e:
    print(f"FAILURE: spaCy GPU check failed: {e}")

print("-" * 40)
"""
    with open("verify.py", "w") as f:
        f.write(verify_script_content)
    print("--- Verification script 'verify.py' created ---")

    # --- Step 4: Execute the script with the injected LD_LIBRARY_PATH ---
    print("\n--- Running verification script ---")
    run_command = f"LD_LIBRARY_PATH={CUDA_LIB_PATH} uv run python verify.py"
    subprocess.run(run_command, shell=True, check=True)

In [1]:
# 1. Install Rust (for the 'tokenizers' dependency)
!curl --proto '=https' --tlsv1.2 -sSf https://sh.rustup.rs | sh -s -- -y
import os
os.environ['PATH'] += ":/root/.cargo/bin"

# 2. Configure the system's dynamic linker to find CUDA libraries
# This makes the libraries available globally for the session.
!echo "/usr/local/cuda/lib64" >> /etc/ld.so.conf.d/cuda.conf
!ldconfig

# 3. Set up uv and install all packages in one command
!uv python install 3.10
!uv python pin 3.10
!uv init --python 3.10
!uv add -q opennyai "spacy[cuda12x]" "numpy<2" "cupy-cuda12x"

info: downloading installer
info: profile set to 'default'
info: default host triple is x86_64-unknown-linux-gnu
info: syncing channel updates for 'stable-x86_64-unknown-linux-gnu'
info: latest update on 2025-06-26, rust version 1.88.0 (6b00bc388 2025-06-23)
info: downloading component 'cargo'
info: downloading component 'clippy'
info: downloading component 'rust-docs'
 20.1 MiB /  20.1 MiB (100 %)  10.7 MiB/s in  1s         
info: downloading component 'rust-std'
 29.5 MiB /  29.5 MiB (100 %)  14.5 MiB/s in  2s         
info: downloading component 'rustc'
 76.3 MiB /  76.3 MiB (100 %)  15.0 MiB/s in  5s         
info: downloading component 'rustfmt'
info: installing component 'cargo'
info: installing component 'clippy'
info: installing component 'rust-docs'
 20.1 MiB /  20.1 MiB (100 %)   5.0 MiB/s in  3s         
info: installing component 'rust-std'
 29.5 MiB /  29.5 MiB (100 %)  11.0 MiB/s in  2s         
info: installing component 'rustc'
 76.3 MiB /  76.3 MiB (100 %)  11.9 MiB/s 

In [9]:
import subprocess

# Because we used ldconfig, we no longer need to modify the environment here.
# The system now knows where to find the CUDA libraries.
python_code = """
import spacy
import cupy

# This should now work correctly
is_gpu_available = spacy.prefer_gpu()
print(f"spaCy GPU support: {is_gpu_available}")

if is_gpu_available:
    gpu_id = cupy.cuda.runtime.getDevice()
    print(f"✅ spaCy is now using GPU ID: {gpu_id}")
else:
    print("❌ GPU support for spaCy is still not available.")

assert is_gpu_available, "spaCy GPU not available"
"""

# Run the simplified verification script inside the uv environment
completed_process = subprocess.run(["uv", "run", "python", "-c", python_code], capture_output=True, text=True)

print("--- Script Output ---")
print(completed_process.stdout)
if completed_process.stderr:
    print("--- Errors ---")
    print(completed_process.stderr)

--- Script Output ---
spaCy GPU support: False
❌ GPU support for spaCy is still not available.

--- Errors ---
Traceback (most recent call last):
  File "<string>", line 15, in <module>
AssertionError: spaCy GPU not available



In [5]:
!uv run \
  --mount "/usr/local/cuda:/usr/local/cuda" \
  --mount "/usr/local/cuda/lib64:/usr/local/cuda/lib64" \
  python -c "import spacy; print('GPU?', spacy.prefer_gpu())"


error: unexpected argument '--mount' found

  tip: a similar argument exists: '--module'

Usage: uv run --module

For more information, try '--help'.


In [8]:
!uv run python - <<EOF
import spacy, cupy
gpu_ok = spacy.prefer_gpu()
print("spaCy GPU support:", gpu_ok)
if gpu_ok:
    print("✅ Using GPU ID:", cupy.cuda.runtime.getDevice())
else:
    raise RuntimeError("spaCy still can’t see the GPU")
EOF


/bin/bash: line 1: warning: here-document at line 1 delimited by end-of-file (wanted `EOF')
spaCy GPU support: True
✅ Using GPU ID: 0


NameError: name 'EOF' is not defined

In [3]:
!nvcc --version


nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [1]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 8.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.5.1
    Uninstalling fsspec-2025.5.1:
      Successfully uninstalled fsspec-2025.5.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.8.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidi

In [2]:
# from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
# from datasets import load_dataset, DatasetDict
# import pandas as pd
# import evaluate
# import numpy as np

# # Load your CSV
# df = pd.read_csv("/kaggle/input/legalseg/val.csv", keep_default_na=False)

# # Clean column names if needed
# df.columns = [c.strip().lower() for c in df.columns]  # ensure 'text' and 'label'

# # Map labels to IDs
# label_list = [
#     "Facts", "Issue", "Arguments of Petitioner", "Arguments of Respondent",
#     "Reasoning", "Decision", "None"
# ]
# label2id = {label: i for i, label in enumerate(label_list)}
# id2label = {i: label for label, i in label2id.items()}
# df["label"] = df["label"].map(label2id)

# # Split the dataset
# from sklearn.model_selection import train_test_split
# train_df, test_df = train_test_split(df, test_size=0.2, stratify=df["label"], random_state=42)

# # Convert to Hugging Face datasets
# from datasets import Dataset
# train_dataset = Dataset.from_pandas(train_df)
# test_dataset = Dataset.from_pandas(test_df)
# dataset = DatasetDict({"train": train_dataset, "test": test_dataset})

# # Tokenize
# model_name = "law-ai/InLegalBERT"
# tokenizer = AutoTokenizer.from_pretrained(model_name)

# def tokenize(example):
#     return tokenizer(example["text"], truncation=True, padding="max_length", max_length=128)

# dataset = dataset.map(tokenize, batched=True)
# dataset = dataset.rename_column("label", "labels")
# dataset.set_format("torch")

# # Load model
# model = AutoModelForSequenceClassification.from_pretrained(
#     model_name,
#     num_labels=len(label_list),
#     id2label=id2label,
#     label2id=label2id
# )

# # Metrics
# accuracy = evaluate.load("accuracy")
# f1 = evaluate.load("f1")

# def compute_metrics(eval_pred):
#     logits, labels = eval_pred
#     preds = np.argmax(logits, axis=1)
#     return {
#         "accuracy": accuracy.compute(predictions=preds, references=labels)["accuracy"],
#         "f1_macro": f1.compute(predictions=preds, references=labels, average="macro")["f1"]
#     }

# # Training setup
# training_args = TrainingArguments(
#     output_dir="./checkpoints",
#     evaluation_strategy="epoch",
#     save_strategy="epoch",
#     logging_strategy="steps",
#     logging_steps=10,
#     learning_rate=2e-5,
#     per_device_train_batch_size=16,
#     per_device_eval_batch_size=16,
#     num_train_epochs=3,
#     weight_decay=0.01,
#     load_best_model_at_end=True,
#     metric_for_best_model="f1_macro"
# )

# # Trainer
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=dataset["train"],
#     eval_dataset=dataset["test"],
#     compute_metrics=compute_metrics
# )

# # Train
# trainer.train()


2025-07-14 14:57:44.997778: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752505065.208762      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752505065.273301      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


tokenizer_config.json:   0%|          | 0.00/516 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Map:   0%|          | 0/234696 [00:00<?, ? examples/s]

...

config.json:   0%|          | 0.00/671 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/534M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at law-ai/InLegalBERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model.safetensors:   0%|          | 0.00/534M [00:00<?, ?B/s]

TypeError: TrainingArguments.__init__() got an unexpected keyword argument 'evaluation_strategy'

In [ ]:
# df = pd.read_csv("/kaggle/input/legalseg/val.csv", keep_default_na=False)
# df.columns = [c.strip().lower() for c in df.columns]  # ensure 'text' and 'label'

# # Map labels to IDs
# label_list = [
#     "Facts", "Issue", "Arguments of Petitioner", "Arguments of Respondent",
#     "Reasoning", "Decision", "None"
# ]
# label2id = {label: i for i, label in enumerate(label_list)}


In [ ]:
# # Temporary mapping without dropping NaNs
# df["mapped_label"] = df["label"].map(label2id)

# # Show rows where mapping failed
# invalid_rows = df[df["mapped_label"].isna()]
# print("Rows with unmapped labels:\n", invalid_rows[["label"]].value_counts())


In [ ]:
# unmapped_labels = df[~df["label"].isin(label2id.keys())]["label"].unique()
# print("Unmapped labels:", unmapped_labels)


In [ ]:
# df = pd.read_csv("/kaggle/input/legalseg/val.csv", keep_default_na=False)
# df.head()

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: anshsinghal (anshsinghal-bennett-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [2]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import torch
import os

# --- 1. Load and Combine CSVs ---
# Load the CSVs
train_df = pd.read_csv('/kaggle/input/legalseg/train.csv')
test_df = pd.read_csv('/kaggle/input/legalseg/test.csv')
val_df = pd.read_csv('/kaggle/input/legalseg/val.csv')

# Drop unused column and clean up
for df in [train_df, test_df, val_df]:
    df.dropna(inplace=True)
    df.drop("Index", axis=1, inplace=True)
    df.columns = ['text', 'label_str']

# Combine train and test
combined_train_df = pd.concat([train_df, test_df], ignore_index=True)

# Create consistent label mapping from all data
unique_labels = pd.concat([combined_train_df, val_df])['label_str'].unique()
label2id = {label: i for i, label in enumerate(unique_labels)}
id2label = {i: label for label, i in label2id.items()}

# Map labels
combined_train_df['label'] = combined_train_df['label_str'].map(label2id)
val_df['label'] = val_df['label_str'].map(label2id)

# Convert to Hugging Face Dataset
train_dataset = Dataset.from_pandas(combined_train_df)
val_dataset = Dataset.from_pandas(val_df)

# --- 2. Tokenizer and Model ---
model_name = "law-ai/InLegalBERT"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(label2id),
    id2label=id2label,
    label2id=label2id
)

# --- 3. Tokenization ---
def tokenize(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=128)

train_dataset = train_dataset.map(tokenize, batched=True)
val_dataset = val_dataset.map(tokenize, batched=True)

train_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])
val_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])

# --- 4. Metric Computation for WandB logging ---
def compute_metrics(eval_pred):
    labels = eval_pred.label_ids
    preds = eval_pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted', zero_division=0)
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

# --- 5. TrainingArguments ---


2025-07-14 15:49:07.056513: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752508147.242484      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752508147.296870      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


tokenizer_config.json:   0%|          | 0.00/516 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/671 [00:00<?, ?B/s]

...

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at law-ai/InLegalBERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/610829 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/534M [00:00<?, ?B/s]

Map:   0%|          | 0/171658 [00:00<?, ? examples/s]

TypeError: TrainingArguments.__init__() got an unexpected keyword argument 'evaluation_strategy'

In [3]:
training_args = TrainingArguments(
    output_dir="./results/inlegalbert-finetuned-legalseg",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
    push_to_hub=False,
    report_to="wandb",  # Enables wandb logging
    run_name="inlegalbert-legalseg-finetune"
)

# --- 6. Trainer ---
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# --- 7. Train ---
trainer.train()

# --- 8. Save the Fine-tuned Model ---
output_dir = "./final_model/inlegalbert-finetuned-legalseg"
os.makedirs(output_dir, exist_ok=True)
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

print("✅ Training complete. Model saved at:", output_dir)


/tmp/ipykernel_36/1354232618.py:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
# training_args = TrainingArguments(
#     output_dir="./results/inlegalbert-finetuned-legalseg",
#     learning_rate=2e-5,
#     per_device_train_batch_size=16,
#     per_device_eval_batch_size=16,
#     num_train_epochs=3,
#     weight_decay=0.01,
#     eval_strategy="epoch",
#     save_strategy="epoch",
#     logging_dir="./logs",
#     logging_steps=10,
#     load_best_model_at_end=True,
#     push_to_hub=False,
#     report_to="wandb",  # Enables wandb logging
#     run_name="inlegalbert-legalseg-finetune"
# )

# # --- 6. Trainer ---
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_dataset,
#     eval_dataset=val_dataset,
#     tokenizer=tokenizer,
#     compute_metrics=compute_metrics
# )

# # --- 7. Train ---
# trainer.train()

# # --- 8. Save the Fine-tuned Model ---
# output_dir = "./final_model/inlegalbert-finetuned-legalseg"
# os.makedirs(output_dir, exist_ok=True)
# model.save_pretrained(output_dir)
# tokenizer.save_pretrained(output_dir)

# print("✅ Training complete. Model saved at:", output_dir)


In [2]:
# import pandas as pd
# from datasets import Dataset
# from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score, precision_recall_fscore_support
# import torch

# # --- 1. Load and Prepare the Dataset ---
# # Load your data from the CSV file
# try:
#     df = pd.read_csv('/kaggle/input/legalseg/val.csv')  # Change this to your actual CSV path
#     df = df.dropna()
#     df = df.drop("Index", axis=1)
#     df.columns = ['text', 'label_str']  # Rename columns for consistency
# except FileNotFoundError:
#     print("CSV file not found. Using a dummy dataframe for demonstration.")
#     # Dummy data for fallback
#     data = {
#         'text': [
#             "The petitioner argues the contract is invalid due to fraud.",
#             "The court held the agreement was enforceable.",
#             "Respondent claims the delay was unavoidable.",
#             "Facts of the case show negligence by the petitioner."
#         ],
#         'label_str': [
#             "Arguments of Petitioner",
#             "Decision",
#             "Arguments of Respondent",
#             "Facts"
#         ]
#     }
#     df = pd.DataFrame(data)

# # Create label mappings
# unique_labels = df['label_str'].unique()
# label2id = {label: i for i, label in enumerate(unique_labels)}
# id2label = {i: label for label, i in label2id.items()}
# df['label'] = df['label_str'].map(label2id)

# # Convert DataFrame to Hugging Face Dataset
# full_dataset = Dataset.from_pandas(df)

# # Train/test split
# train_ds, test_ds = full_dataset.train_test_split(test_size=0.2, seed=42).values()

# # --- 2. Load Tokenizer and Model ---
# model_name = "law-ai/InLegalBERT"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForSequenceClassification.from_pretrained(
#     model_name,
#     num_labels=len(unique_labels),
#     id2label=id2label,
#     label2id=label2id
# )

# # --- 3. Tokenization ---
# def tokenize_function(examples):
#     return tokenizer(examples["text"], padding="max_length", truncation=True)

# tokenized_train_ds = train_ds.map(tokenize_function, batched=True)
# tokenized_test_ds = test_ds.map(tokenize_function, batched=True)

# # Set format for PyTorch
# tokenized_train_ds.set_format("torch", columns=["input_ids", "attention_mask", "label"])
# tokenized_test_ds.set_format("torch", columns=["input_ids", "attention_mask", "label"])

# # --- 4. Define Training Configuration ---
# def compute_metrics(pred):
#     labels = pred.label_ids
#     preds = pred.predictions.argmax(-1)
#     precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
#     acc = accuracy_score(labels, preds)
#     return {
#         'accuracy': acc,
#         'f1': f1,
#         'precision': precision,
#         'recall': recall
#     }

# training_args = TrainingArguments(
#     output_dir="./results/inlegalbert-finetuned-legalseg",
#     learning_rate=2e-5,
#     per_device_train_batch_size=16,
#     per_device_eval_batch_size=16,
#     num_train_epochs=3,
#     weight_decay=0.01,
#     eval_strategy="epoch",
#     save_strategy="epoch",
#     load_best_model_at_end=True,
#     push_to_hub=False,
#     logging_dir="./logs",
#     logging_steps=10
# )

# # --- 5. Train the Model ---
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=tokenized_train_ds,
#     eval_dataset=tokenized_test_ds,
#     tokenizer=tokenizer,
#     compute_metrics=compute_metrics,
# )

# # Train the model
# trainer.train()

# # Save the fine-tuned model
# model.save_pretrained("./final_model/inlegalbert-finetuned-legalseg")
# tokenizer.save_pretrained("./final_model/inlegalbert-finetuned-legalseg")

# print("Training complete. Model saved.")


2025-07-14 15:29:36.305712: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752506976.495074      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752506976.547371      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


tokenizer_config.json:   0%|          | 0.00/516 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/671 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/534M [00:00<?, ?B/s]

Map:   0%|          | 0/34332 [00:00<?, ? examples/s]

/tmp/ipykernel_36/1204273043.py:96: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
import pandas as pd
from datasets import Dataset
from transformers import (
    AutoTokenizer, 
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer
)
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import os
import wandb

# --- 1. Load and Combine CSVs ---
train_df = pd.read_csv('/kaggle/input/legalseg/train.csv')
test_df = pd.read_csv('/kaggle/input/legalseg/test.csv')
val_df = pd.read_csv('/kaggle/input/legalseg/val.csv')

for df in [train_df, test_df, val_df]:
    df.dropna(inplace=True)
    df.drop("Index", axis=1, inplace=True)
    df.columns = ['text', 'label_str']

# Combine train and test
combined_train_df = pd.concat([train_df, test_df], ignore_index=True)

# Label mapping
unique_labels = pd.concat([combined_train_df, val_df])['label_str'].unique()
label2id = {label: i for i, label in enumerate(unique_labels)}
id2label = {i: label for label, i in label2id.items()}
combined_train_df['label'] = combined_train_df['label_str'].map(label2id)
val_df['label'] = val_df['label_str'].map(label2id)

train_dataset = Dataset.from_pandas(combined_train_df)
val_dataset = Dataset.from_pandas(val_df)

# --- 2. Load Tokenizer and Model ---
model_name = "law-ai/InLegalBERT"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(label2id),
    id2label=id2label,
    label2id=label2id
)

# --- 3. Tokenize ---
def tokenize(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=256)

train_dataset = train_dataset.map(tokenize, batched=True)
val_dataset = val_dataset.map(tokenize, batched=True)
train_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])
val_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])

# --- 4. Metric Function ---
def compute_metrics(eval_pred):
    labels = eval_pred.label_ids
    preds = eval_pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted', zero_division=0)
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

# --- 5. Training Arguments ---
training_args = TrainingArguments(
    output_dir="./results/inlegalbert-finetuned-legalseg",
    learning_rate=1e-6,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=10,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    logging_dir="./logs",
    logging_steps=10,
    report_to="wandb",              # Log to wandb
    run_name="inlegalbert-legalseg-finetune"
)

# --- 6. Trainer ---
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# --- 7. Train ---
trainer.train()

# --- 8. Save Best Model to Directory ---
output_dir = "./final_model/inlegalbert-finetuned-legalseg"
os.makedirs(output_dir, exist_ok=True)
trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)

# --- 9. Upload Best Model as Artifact to W&B ---
artifact = wandb.Artifact(name="inlegalbert-legalseg-best", type="model")
artifact.add_dir(output_dir)
wandb.log_artifact(artifact)

print("✅ Training complete. Best model saved and logged to W&B.")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at law-ai/InLegalBERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/610829 [00:00<?, ? examples/s]

Map:   0%|          | 0/171658 [00:00<?, ? examples/s]

/tmp/ipykernel_36/1754307305.py:88: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss


In [45]:
import os
import json
from glob import glob
from opennyai import Pipeline
from opennyai.utils import Data

In [43]:
!pip install spacy-transformers
!pip install https://huggingface.co/opennyaiorg/en_legal_ner_trf/resolve/main/STOCK_SPACY_MODELS/en_core_web_trf-3.2.0-py3-none-any.whl --no-deps


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 460.2/460.2 MB 71.3 MB/s eta 0:00:0000:0100:01


In [40]:
 pip install -U https://huggingface.co/opennyaiorg/en_legal_ner_trf/resolve/main/STOCK_SPACY_MODELS/en_core_web_trf-3.2.0-py3-none-any.whl --no-deps

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 460.2/460.2 MB 75.6 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [39]:
!pip install -U pip setuptools wheel
!pip install tokenizers>=0.14.1 transformers


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [30]:
# Install transformers & tokenizers manually
!pip install -U 'transformers==4.20.1' 'tokenizers==0.12.1' --prefer-binary --no-build-isolation

# Then install opennyai again (should skip transformers/tokenizers now)
!pip install -U opennyai --no-deps


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached transformers-4.20.1-py3-none-any.whl.metadata (77 kB)
  Using cached tokenizers-0.12.1.tar.gz (220 kB)
  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (pyproject.toml) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [26]:
 pip install -U https://huggingface.co/opennyaiorg/en_legal_ner_trf/resolve/main/STOCK_SPACY_MODELS/en_core_web_trf-3.2.0-py3-none-any.whl

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 460.2/460.2 MB 3.6 MB/s eta 0:00:0000:0100:01
  Using cached spacy-3.2.6-cp311-cp311-linux_x86_64.whl
  Using cached spacy_transformers-1.1.9-py2.py3-none-any.whl.metadata (6.2 kB)
  Using cached thinc-8.0.17-cp311-cp311-linux_x86_64.whl
  Using cached blis-0.7.11-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.4 kB)
  Using cached pydantic-1.8.2-py3-none-any.whl.metadata (103 kB)
  Using cached typing_extensions-4.5.0-py3-none-any.whl.metadata (8.5 kB)
INFO: pip is looking at multiple versions of spacy-transformers to determine which version is compatible with other requirements. This could take a while.
  Using cached spacy_transformers-1.1.8-py2.py3-none-any.whl.metadata (6.1 kB)
  Using cached spacy_transformers-1.1.7-py2.py3-none-any.whl.metadata (6.0 kB)
  Using cached transformers-4.20.1-py3-none-any.whl.metadata (77 kB)
  Using cached tokenizers-0.12.1.tar.gz (220 kB)
  Installing build dependencies ... done
  

In [24]:
!pip install scikit-learn==1.2.2 --force-reinstall


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached numpy-2.3.1-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (62 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 70.8 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 22.4 MB/s eta 0:00:00
Using cached numpy-2.3.1-cp311-cp311-manylinux_2_28_x86_64.whl (16.9 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.3/35.3 MB 47.7 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: threadpoolctl
    Found existing installation: threadpoolctl 3.6.0
    Uninstalling threadpoolctl-3.6.0:
      Successfully uninstalled threadpoolctl-3.6.0
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.6
    Uninstalling numpy-2.2.6:
      Successfully uninstalled numpy-2.2.6
  Attempting uninstall: joblib
    Found existing installation: joblib 1.5.1
    Uninstalling joblib-1.5.1:
      Successfully uninstalled joblib-1.5.1
  Attempting uninstall: sci

In [35]:
pip install -U pip setuptools wheel


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [36]:
pip install tokenizers>=0.14.1 transformers


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [37]:
!pip install -U opennyai


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached scikit_learn-1.7.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (17 kB)
  Using cached spacy-3.2.6-cp311-cp311-linux_x86_64.whl
  Using cached thinc-8.0.17-cp311-cp311-linux_x86_64.whl
  Using cached blis-0.7.11-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.4 kB)
  Using cached pydantic-1.8.2-py3-none-any.whl.metadata (103 kB)
  Using cached typing_extensions-4.5.0-py3-none-any.whl.metadata (8.5 kB)
INFO: pip is looking at multiple versions of spacy-transformers to determine which version is compatible with other requirements. This could take a while.
  Using cached spacy_transformers-1.3.8-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.0 kB)
  Using cached spacy_transformers-1.3.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.0 kB)
  Using cached spacy_transformers-1.3.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.0 kB)
  Using cached spacy_transfo

In [15]:
!pip install opennyai --no-deps  # Avoid pulling old deps that break


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached opennyai-0.0.13-py3-none-any.whl.metadata (6.2 kB)
Using cached opennyai-0.0.13-py3-none-any.whl (170 kB)


In [21]:
!pip install \
  "spacy>=3.2.2,<3.3.0" \
  "pandas>1.2.4" \
  "beautifulsoup4>=4.10.0" \
  "torch>=1.12.1" \
  "levenshtein==0.23.0" \
  "multiprocess==0.70.15" \
  "scikit-learn==1.3.2" \
  "pytest==7.4.3" \
  "prettytable>=3.1.1" \
  "nltk>=3.6" \
  "pytorch-transformers==1.2.0"


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached spacy-3.2.6-cp311-cp311-linux_x86_64.whl
  Using cached Levenshtein-0.23.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.4 kB)
  Using cached multiprocess-0.70.15-py311-none-any.whl.metadata (7.2 kB)
  Using cached scikit_learn-1.3.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
  Using cached pytest-7.4.3-py3-none-any.whl.metadata (7.9 kB)
  Using cached pytorch_transformers-1.2.0-py3-none-any.whl.metadata (21 kB)
  Using cached rapidfuzz-3.13.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (12 kB)
  Using cached sacremoses-0.1.1-py3-none-any.whl.metadata (8.3 kB)
  Using cached thinc-8.0.17-cp311-cp311-linux_x86_64.whl
  Using cached blis-0.7.11-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.4 kB)
  Using cached wasabi-0.10.1-py3-none-any.whl.metadata (28 kB)
  Using cached typer-0.4.2-py3-none-any.whl.metadata (12 kB)
  Using cached pathy-0.11.0-py3-none-any.whl.metada

In [22]:
!pip install spacy-transformers>=1.1.4 --prefer-binary



huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.8.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
scikit-learn 1.3.2 requires numpy<2.0,>=1.17.3, but you have numpy 2.2.6 which is incompatible.
opennyai 0.0.13 requires spacy<3.3.0,>=3.2.2, but you have spacy 3.8.7 which is incompatible.
gensim 4.3.3 requires numpy<2.0,>=1.18.5, but you have numpy 2.2.6 which is incompatible.
gensim 4.3.3 requires scipy<1.14.0,>=1.7.0, but you have scipy 1.15.3 which is incompatible.
mkl-umath 0.1.1 requires numpy<1.27.0,>=1.26.4, but you have numpy 2.2.6 which is incompatible.
mkl-random 1.2.4 requires numpy<1.27.0,>=1.26.4, but you have numpy 2.2.6 which is incompatible.
mkl-fft 1.3.8 requires numpy<1.27.0,>=1.26.4, but you have numpy 2.2.6 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.6 which

In [10]:
!pip install cupy

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 41.5 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for cupy (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for cupy
Failed to build cupy
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (cupy)


In [8]:
!nvcc --version


nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [47]:
!ls /kaggle/working/DHARA/rrl

In [48]:
with open("/kaggle/working/pyproject.toml", 'r') as f:
    all = f.read()
print(all)

[project]
name = "working"
version = "0.1.0"
description = "Add your description here"
readme = "README.md"
requires-python = ">=3.10"
dependencies = [
    "beautifulsoup4>=4.10.0",
    "cupy-cuda12x>=13.5.1",
    "levenshtein==0.23.0",
    "multiprocess==0.70.15",
    "nltk>=3.6",
    "opennyai>=0.0.13",
    "pandas>1.2.4",
    "prettytable>=3.1.1",
    "pytest==7.4.3",
    "pytorch-transformers==1.2.0",
    "scikit-learn==1.3.2",
    "spacy>=3.2.2,<3.3.0",
    "spacy-transformers>=1.1.4,<1.2.0",
    "torch>=1.12.1",
]



In [61]:
!rm /kaggle/working/uv.lock
!rm /kaggle/working/pyproject.toml
!ls /kaggle/working/

DHARA  main.py	README.md  test.py


In [21]:
!nvcc --version


nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [1]:
!uv python install 3.10
!uv python pin 3.10
!uv init --python 3.10
!uv add -q opennyai
!uv add -q cupy-cuda12x
# Recommended: Install spaCy with CUDA auto-detection
!uv add "spacy[cuda-autodetect]"
!uv run python --version



Installed Python 3.10.16 in 1.76s------------------- 19.77 MiB/19.81 MiB                                                                             
 + cpython-3.10.16-linux-x86_64-gnu
Pinned `.python-version` to `3.10`
Initialized project `working`
Resolved 97 packages in 60ms                                         
Audited 92 packages in 0.03ms
Python 3.10.16


In [2]:
!uv run pip show spacy


Name: spacy
Version: 3.8.7
Summary: Industrial-strength Natural Language Processing (NLP) in Python
Home-page: https://spacy.io
Author: Explosion
Author-email: contact@explosion.ai
License: MIT
Location: /usr/local/lib/python3.11/dist-packages
Requires: catalogue, cymem, jinja2, langcodes, murmurhash, numpy, packaging, preshed, pydantic, requests, setuptools, spacy-legacy, spacy-loggers, srsly, thinc, tqdm, typer, wasabi, weasel
Required-by: fastai


In [ ]:
# Install the Rust compiler and its package manager, Cargo
!curl --proto '=https' --tlsv1.2 -sSf https://sh.rustup.rs | sh -s -- -y

import os
os.environ['PATH'] += ":/root/.cargo/bin"

# !uv add -q "numpy<2"
!pip install opennyai
!pip install cupy-cuda12x
!pip install -U "spacy[cuda12x]"


In [ ]:
# 1. Install Rust (for the 'tokenizers' dependency)
!curl --proto '=https' --tlsv1.2 -sSf https://sh.rustup.rs | sh -s -- -y
import os
os.environ['PATH'] += ":/root/.cargo/bin"

# 2. Configure the system's dynamic linker to find CUDA libraries
# This makes the libraries available globally for the session.
!echo "/usr/local/cuda/lib64" >> /etc/ld.so.conf.d/cuda.conf
!ldconfig

# 3. Set up uv and install all packages in one command
!uv python install 3.10
!uv python pin 3.10
!uv init --python 3.10
!uv add -q opennyai "spacy[cuda12x]" "numpy<2"

In [5]:

import os
import subprocess

# Define the Python code you want to run
# Crucially, we set the LD_LIBRARY_PATH inside the code being executed
python_code = """
import os
# Point to the location of Kaggle's CUDA shared libraries
os.environ['LD_LIBRARY_PATH'] = '/usr/local/cuda/lib64'

import spacy
import cupy

# This should now return True
is_gpu_available = spacy.prefer_gpu()
print(f"spaCy GPU support: {is_gpu_available}")

if is_gpu_available:
    gpu_id = cupy.cuda.runtime.getDevice()
    print(f"✅ spaCy is now using GPU ID: {gpu_id}")
else:
    print("❌ GPU support for spaCy is still not available.")

assert is_gpu_available, "spaCy GPU not available"
"""

# Use `uv run` to execute the code within the virtual environment
# The `-c` flag allows passing the code as a string
completed_process = subprocess.run(["uv", "run", "python", "-c", python_code], capture_output=True, text=True)

# Print the output and any errors
print("--- Script Output ---")
print(completed_process.stdout)
if completed_process.stderr:
    print("--- Errors ---")
    print(completed_process.stderr) 

--- Script Output ---
spaCy GPU support: False
❌ GPU support for spaCy is still not available.

--- Errors ---
Traceback (most recent call last):
  File "<string>", line 19, in <module>
AssertionError: spaCy GPU not available



In [7]:

code = """
import spacy
import cupy

# This will now set spaCy to use the GPU and return True
is_gpu_available = spacy.prefer_gpu()

print(f"spaCy GPU support: {is_gpu_available}")

if is_gpu_available:
    # Get the ID of the current GPU device used by CuPy
    gpu_id = cupy.cuda.runtime.getDevice()
    print(f"spaCy is using GPU: {gpu_id}")
else:
    print("GPU not available for spaCy. Check your CUDA installation and package versions.")

# This assertion should now pass without error
assert is_gpu_available, "spaCy GPU is still not available."

import sys

print("Python version:", sys.version)
print("Version info:", sys.version_info)

import cupy
print("cupy.cuda.runtime.getDeviceCount()")

print(cupy.cuda.runtime.getDeviceCount())

import torch
print("CUDA available:", torch.cuda.is_available())

import spacy
# from spacy.util import use_gpu

print("spaCy GPU support:", spacy.prefer_gpu())
# print("spaCy is using GPU:", use_gpu())
import spacy
assert spacy.prefer_gpu() == True, "spaCy GPU not available"
print("✅ spaCy GPU is available")


"""

with open("temp.py", "w") as f:
    f.write(code)

!uv run temp.py


spaCy GPU support: False
GPU not available for spaCy. Check your CUDA installation and package versions.
Traceback (most recent call last):
  File "/kaggle/working/temp.py", line 18, in <module>
    assert is_gpu_available, "spaCy GPU is still not available."
AssertionError: spaCy GPU is still not available.


In [2]:
import spacy
import cupy

# This will now set spaCy to use the GPU and return True
is_gpu_available = spacy.prefer_gpu()

print(f"spaCy GPU support: {is_gpu_available}")

if is_gpu_available:
    gpu_id = cupy.cuda.runtime.getDevice()
    print(f"✅ spaCy is now using GPU ID: {gpu_id}")
else:
    print("❌ GPU support for spaCy is still not available. Check the installation logs.")

# This assertion should now pass without error
assert is_gpu_available, "spaCy GPU not available"

spaCy GPU support: True
✅ spaCy is now using GPU ID: 0


In [ ]:
!pip install opennyai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 33.3 MB/s eta 0:00:00


In [ ]:
!pip install uv

!uv python install 3.10

!uv init --python 3.10

!uv python pin 3.10

# !uv venv

# STEP 1: Create or upgrade virtual env (already done)
# !python3 -m venv /kaggle/working/.venv
# !/kaggle/working/.venv/bin/python3 -m ensurepip --upgrade

# STEP 2: Activate the venv
# !source .venv/bin/activate
# !source /kaggle/working/.venv/bin/activate

!uv pip install "spacy>=3.2.2,<3.3.0" "pandas>1.2.4" "beautifulsoup4>=4.10.0" "torch>=1.12.1" "Levenshtein==0.23.0" "multiprocess==0.70.15" "spacy-transformers>=1.1.4,<1.2.0" "scikit-learn==1.3.2" "pytest==7.4.3" "prettytable>=3.1.1" "nltk>=3.6" "pytorch-transformers==1.2.0" "opennyai"


# STEP 3: Add required CUDA-enabled packages
!uv add cupy-cuda12x

# STEP 4: Install spaCy with CUDA support
!uv pip install -U "spacy[cuda12x]"

# STEP 5: Install spaCy-transformers and OpenNyAI

# !uv pip install spacy-transformers opennyai

# STEP 6: Download the transformer-enabled spaCy model
!/kaggle/working/.venv/bin/python3 -m spacy download en_core_web_trf


In [14]:
!git clone https://github.com/AnshSinghal/DHARA.git


fatal: destination path 'DHARA' already exists and is not an empty directory.


In [15]:
code = """
import os
import json
from glob import glob
from opennyai import Pipeline
from opennyai.utils import Data

import spacy
spacy.require_gpu()

# Paths
input_dir = "/kaggle/working/DHARA/cases"
output_dir = "/kaggle/working/DHARA/rrl"
os.makedirs(output_dir, exist_ok=True)

# 1. Collect and read all .txt case files
file_paths = sorted(glob(os.path.join(input_dir, "*.txt")))
texts = []
doc_ids = []

for path in file_paths:
    with open(path, 'r', encoding='utf-8') as f:
        texts.append(f.read())
        doc_ids.append(os.path.splitext(os.path.basename(path))[0])

# 2. Wrap with OpenNyAI-compatible Data object
data = Data(texts, use_gpu=True)

# 3. Init pipeline (use GPU if available)
pipe = Pipeline(components=["Rhetorical_Role"], use_gpu=True)

# 4. Run RRL
results = pipe(data)

# 5. Save each result as individual JSON file
for doc_id, result in zip(doc_ids, results):
    output_path = os.path.join(output_dir, f"{doc_id}.json")
    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(result, f, ensure_ascii=False, indent=2)

print(f"✅ Inference complete. {len(results)} files saved to {output_dir}")
"""

with open("DHARA/rrl_run.py", "w") as f:
    f.write(code)


In [ ]:
!/kaggle/working/.venv/bin/python3 -m ensurepip --upgrade


!nvcc --version

!uv add cupy-cuda12x


!uv pip install "spacy[cuda12x]"


!uv run DHARA/rrl_run.py
